The objective of this notebook is transforming a folder filled with images into FGSM-modified cover images following the approaches shown in the repository.

## Imports

In [13]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.models import load_model
import os
import numpy as np
import glob

mpl.rcParams['figure.figsize'] = (8, 8)
mpl.rcParams['axes.grid'] = False

# Fast Gradient Sign Method (FGSM)

FGSM is a white box attack is that one that is performed by an attacker that has access to the whole model.

This method was presented in the [Explaining and Harnessing Adversarial Examples by Ian Goodfellow et al.](https://arxiv.org/abs/1412.6572). It consists on using the gradients of the loss with respect to the input image to create a new image that maximizes the loss. The new image generated is the adversarial image. The following expression summarizes everything:

\begin{align}
adv_{-} x=x+\epsilon * \operatorname{sign}\left(\nabla_{x} J(\theta, x, y)\right)
\end{align}

The gradients are taken with respect to the input image because the objective is to check how much each pixel of the image contributes to the loss value.

As mentioned in the title of this Notebook, this is a white box attack. Some other [more advanced approaches](https://arxiv.org/abs/1602.02697) have been taken using FGSM from a Black Box perspective. However, this is much more complex.

The code used below is based in [the implementation of the Official Tensorflow Page](https://www.tensorflow.org/tutorials/generative/adversarial_fgsm).

## Constants

The model used in this notebook is the available in the [SRNet-Tensorflow-Implementation](https://github.com/davidggz/SRNet-Tensorflow-Implementation) repository.

In [14]:
MODEL_PATH = 'model_checkpoint/best_model.h5'
IMAGE_DIR = 'Path to the folder with cover and stego images'
OUTPUT_DIR = 'Name or path of the output folder'
EPSILON = 0.01

# Loss that will be used to calculate the gradient. This loss must change 
# in case softmax is used as the output activation function.
LOSS_OBJECT = tf.keras.losses.BinaryCrossentropy(from_logits=False)

## Load the model to be attacked

In [15]:
# Load the model
model = load_model(MODEL_PATH)

# Don't allow training the model
model.trainable = False

## Main functions

In [16]:
def preprocessing(image):
    # Scale the image so that it's in the range 0-1.
    image = image * 1./255
    return image


def read_and_preprocess_image(image_filename):
    # Read the image not changing any pixel
    image = cv2.imread(image_filename, cv2.IMREAD_UNCHANGED)

    # Expand the dimensions to have a shape (1, N, M, 1)
    image = np.expand_dims(image, axis=(0, 3))
    
    # Preprocess the image
    preprocessed_image = preprocessing(image)
    
    return preprocessed_image
    

# This is just a FGSM implementation
def create_adversarial_pattern(input_image, input_label):
    ''' This function receives a [-1, 1] image and it returns 
    the direction that we need to go in each pixel in order to 
    go far from the input_label
    '''
    # Transform the input image into a tensor
    tf_image = tf.convert_to_tensor(input_image, dtype=tf.float32)
    
    # Expand the dimensions to have shape (1, 1)
    input_label = np.expand_dims(input_label, axis=(0, 1))
    
    # Initialize GradientTape store the interaction of the input with the model
    with tf.GradientTape() as tape:
        # Indicate that we want to be aware of tf_image
        tape.watch(tf_image)
        
        # Make the prediction and get the loss
        prediction = model(tf_image)
        loss = LOSS_OBJECT(input_label, prediction)

    # Once we have the loss and we've stored in the tape everything that happened during the inference,
    # we can check the gradient with respect to any variable or constant. In this case,
    # the gradient is obtained with respecto to the image itself.
    gradient = tape.gradient(loss, tf_image)
    
    # We get just the sign of the gradient.
    signed_grad = tf.sign(gradient)
    
    return signed_grad

## fromCover algorithm

As it is stated in the repository description, this algorithm takes the cover images and it generates its FGSM-modified versions by using the cover image itself as the input of the FGSM algorithm. Once the FGSM-modified images have been generated, it is necessary to introduce the stego message into the images. These images will be prepared to attack a determined SRNet model.

In [17]:
# Generate the folders necessary to train and test.
assert not os.path.isdir(OUTPUT_DIR)
os.mkdir(OUTPUT_DIR)

sets = ['train', 'val']
for set_name in sets:
    os.mkdir(os.path.join(OUTPUT_DIR, set_name))
    os.mkdir(os.path.join(OUTPUT_DIR, set_name, '0'))
    os.mkdir(os.path.join(OUTPUT_DIR, set_name, '1'))

In [18]:
# Get all the images to transform
all_original_fullpaths = glob.glob(os.path.join(IMAGE_DIR, '*', '*',  '*'))

# Transform every image in the directory
for original_fullpath in all_original_fullpaths:
    # Get the input label from the name of the directory
    input_label = int(original_fullpath.split('\\')[-2])
    
    # Read and preprocess the image
    preprocessed_image = read_and_preprocess_image(original_fullpath)
    
    # Change the image to be in the range [-1, 1]
    preprocessed_image = preprocessed_image * 2 - 1
    
    # Get the perturbations
    perturbations = create_adversarial_pattern(preprocessed_image, input_label)
    
    # Transform the input image with the perturbations
    adversarial_image = preprocessed_image + EPSILON * perturbations
    
    # Remove those values that are below -1 or above 1
    adversarial_image = tf.clip_by_value(adversarial_image, -1, 1)
    
    # Transform the image back again into the range [0, 255]
    adversarial_image = (adversarial_image * 0.5 + 0.5) * 255
    
    # Transform to numpy and reshape
    adversarial_image = np.reshape(adversarial_image.numpy(), (256, 256))
    
    # Store the image in OUTPUT_DIR
    out_fullpath = os.path.join(OUTPUT_DIR, original_fullpath.split('\\')[-3], str(input_label), original_fullpath.split('\\')[-1])
    cv2.imwrite(out_fullpath, adversarial_image)

## fromStego algorithm

This is an algorithm similar to the previous one. However, in this case the stego images are necessary since the FGSM perturbations to be applied to the cover images are obtained from the stego images. Once the cover images have been modified, the hidden message has to be added again to the FGSM-modified images.

In [17]:
# Create the output directory
assert not os.path.isdir(OUTPUT_DIR)
os.mkdir(OUTPUT_DIR)

os.mkdir(os.path.join(OUTPUT_DIR, '0'))
os.mkdir(os.path.join(OUTPUT_DIR, '1'))

In [18]:
# Get all the images to transform
cover_full_filenames = glob.glob(os.path.join(IMAGE_DIR, '0', '*'))
stego_full_filenames = glob.glob(os.path.join(IMAGE_DIR, '1', '*'))

# Transform every image in the directory
for cover_full_filename, stego_full_filename in zip(cover_full_filenames, stego_full_filenames):
    # In this case the input label is always 1 because we are obtaining
    # the perturbations from the stego images.
    input_label = 1
    
    # Read and preprocess the image
    cover_preprocessed_image = read_and_preprocess_image(cover_full_filename)
    stego_preprocessed_image = read_and_preprocess_image(stego_full_filename)
    
    # Change the image to be in the range [-1, 1]
    cover_preprocessed_image = cover_preprocessed_image * 2 - 1
    stego_preprocessed_image = stego_preprocessed_image * 2 - 1
    
    # Get the perturbations
    perturbations = create_adversarial_pattern(stego_preprocessed_image, input_label)
    
    # Transform the input image with the perturbations
    adversarial_image = cover_preprocessed_image + EPSILON * perturbations
    
    # Remove those values that are below -1 or above 1
    adversarial_image = tf.clip_by_value(adversarial_image, -1, 1)
    
    # Transform the image back again into the range [0, 255]
    adversarial_image = (adversarial_image * 0.5 + 0.5) * 255
    
    # Transform to numpy and reshape
    adversarial_image = np.reshape(adversarial_image.numpy(), (256, 256))
    
    # Store the image in OUTPUT_DIR
    out_fullpath = os.path.join(OUTPUT_DIR, '0', cover_full_filename.split('\\')[-1])
    cv2.imwrite(out_fullpath, adversarial_image)